In [1]:
import pandas as pd
import numpy as np

from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

from data_pipeline.pipelinesteps import data_splitter
from model.modelorchastrator import ModelOrchestrator
from data_pipeline.dataset import Dataset
from evaluate.metric_eval import ModelEvaluator

(CVXPY) Apr 02 10:35:44 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 02 10:35:44 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
 2024-04-02 10:35:45,067 -  INFO  	: my_logger.custom_logger - Logger is set up.


In [2]:
# Create config
GlobalHydra.instance().clear()
initialize(config_path="../../conf", version_base=None)
cfg = compose(config_name="config")

In [3]:
# Create Dataset
df = pd.read_feather("../../data/parquet_files/train/processed_train.feather")
df.pop('date_decision')
data_pipeline = instantiate(cfg.data_pipeline)
df = data_pipeline.apply(df)  

dataset = Dataset(data=df, data_splitter=data_splitter, target_column='target')

In [4]:
model_orchestrator = ModelOrchestrator(cfg)

model = model_orchestrator.modelpipeline
ME = ModelEvaluator(model[-1], ['roc_auc_ovr', 'f1'])

In [5]:
cfg.model.model_steps[1]

{'OptBinningTransformer': {'_target_': 'model.modelorchastrator.OptBinningTransformer', 'variable_names': ['housetype_905L_last', 'education_927M_last', 'credacc_cards_status_52L_mode', 'familystate_447L_last', 'credacc_cards_status_52L_last', 'classificationofcontr_400M_mode', 'credtype_322L_last', 'cardtype_51L_last', 'cardtype_51L_mode'], 'max_n_prebins': 346, 'min_prebin_size': 0.04717159150979889}}

In [6]:
from omegaconf import DictConfig, OmegaConf
t = cfg.model.model_steps[1]
next(iter(t.items()))[1]

{'_target_': 'model.modelorchastrator.OptBinningTransformer', 'variable_names': ['housetype_905L_last', 'education_927M_last', 'credacc_cards_status_52L_mode', 'familystate_447L_last', 'credacc_cards_status_52L_last', 'classificationofcontr_400M_mode', 'credtype_322L_last', 'cardtype_51L_last', 'cardtype_51L_mode'], 'max_n_prebins': 346, 'min_prebin_size': 0.04717159150979889}

In [7]:
df['education_927M_last']  

0          a55475b1
1          a55475b1
2          a55475b1
3          a55475b1
4          a55475b1
             ...   
1526654    a55475b1
1526655    a55475b1
1526656    a55475b1
1526657    a55475b1
1526658    a55475b1
Name: education_927M_last, Length: 1526659, dtype: object

In [8]:
model.fit(dataset.X_train, dataset.y_train)

 2024-04-02 10:35:48,356 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


CustomModelPipeline(steps=[('0',
                            FeatureSelector(features=['childnum_185L_mean', 'housetype_905L_last', 'familystate_447L_last', 'education_927M_last', 'cardtype_51L_last', 'credtype_322L_last', 'numberofcontrsvalue_358L_max', 'nominalrate_281L_min', 'totaldebtoverduevalue_178A_max', 'overdueamount_31A_max', 'residualamount_488A_max', 'collater_valueofguarante...
                                                  variable_names=['housetype_905L_last', 'education_927M_last', 'credacc_cards_status_52L_mode', 'familystate_447L_last', 'credacc_cards_status_52L_last', 'classificationofcontr_400M_mode', 'credtype_322L_last', 'cardtype_51L_last', 'cardtype_51L_mode'])),
                           ('2',
                            HistGradientBoostingClassifier(class_weight='balanced',
                                                           early_stopping=True,
                                                           l2_regularization=1.78010903230611,
                                                           learning_rate=0.199948962341622,
                                                           max_bins=206,
                                                           max_depth=990,
                                                           max_iter=2696,
                                                           max_leaf_nodes=96))])

In [9]:
model.transform_without_predictor(dataset.X_train)

 2024-04-02 10:35:55,842 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


,childnum_185L_mean,housetype_905L_last,familystate_447L_last,education_927M_last,cardtype_51L_last,credtype_322L_last,numberofcontrsvalue_358L_max,nominalrate_281L_min,totaldebtoverduevalue_178A_max,overdueamount_31A_max,...,last180dayaveragebalance_704A_last,last180dayaveragebalance_704A_mean,last180dayaveragebalance_704A_max,classificationofcontr_400M_mode,numinstlswithdpd5_4187116L_last,numinstlsallpaid_934L_last,numinstls_657L_last,cardtype_51L_mode,avginstallast24m_3658937A_last,mainoccupationinc_437A_last
221249,NaN,0.0,0.0,2.220446e-16,0.000000,-0.003175,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,NaN,NaN,0.0,0.000000,NaN,NaN
1220007,NaN,0.0,0.0,2.220446e-16,0.000000,-0.003175,8.0,NaN,0.0,0.0,...,NaN,NaN,NaN,0.09945,9.0,14.0,0.0,0.000000,NaN,5800.0
1168023,NaN,0.0,0.0,2.220446e-16,0.000000,-0.003175,12.0,39.0,0.0,0.0,...,NaN,NaN,NaN,0.09945,0.0,29.0,12.0,0.000000,1553.2001,26000.0
145230,NaN,0.0,0.0,2.220446e-16,0.000000,-0.083811,4.0,42.0,0.0,0.0,...,NaN,NaN,NaN,0.09945,0.0,24.0,33.0,0.000000,12923.2000,199600.0
338605,NaN,0.0,0.0,2.220446e-16,0.120749,0.122728,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,NaN,NaN,0.0,0.120749,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222458,NaN,0.0,0.0,2.220446e-16,0.000000,-0.003175,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,NaN,NaN,0.0,0.000000,NaN,NaN
138401,NaN,0.0,0.0,2.220446e-16,0.120749,0.122728,8.0,4.8,0.0,0.0,...,NaN,NaN,NaN,0.09945,1.0,36.0,42.0,0.120749,13893.0000,10000.0
183208,NaN,0.0,0.0,2.220446e-16,0.000000,-0.083811,1.0,NaN,0.0,0.0,...,NaN,NaN,NaN,0.09945,0.0,6.0,6.0,0.000000,1749.8000,24000.0
222110,NaN,0.0,0.0,2.220446e-16,0.000000,-0.003175,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00000,NaN,NaN,0.0,0.000000,NaN,NaN


In [10]:
ME.evaluate(model.transform_without_predictor(dataset.X_train), dataset.y_train)

 2024-04-02 10:35:57,106 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


array([0.74791505, 0.75382539, 0.7493069 , 0.74713551, 0.74793727])

In [11]:
from sklearn.metrics import get_scorer_names
all_scorers = get_scorer_names()
all_scorers

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'neg_root_mean_squared_log_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 '

In [12]:
from sklearn.metrics import roc_auc_score

def gini(y_true, y_pred):
    return 2 * roc_auc_score(y_true, y_pred) - 1

In [13]:
gini(dataset.y_train, model.predict_proba(dataset.X_train).T[1])



 2024-04-02 10:36:22,788 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.5668578763095531

In [14]:
import numpy as np

std_dev = np.std(model.predict_proba(dataset.X_train).T[1])
print(std_dev)


 2024-04-02 10:36:25,394 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.2040960965586104


In [15]:
roc_auc_score(dataset.y_train, model.predict_proba(dataset.X_train).T[1])

 2024-04-02 10:36:27,787 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.7834289381547765

In [16]:
roc_auc_score(dataset.y_oot, model.predict_proba(dataset.X_oot).T[1])

 2024-04-02 10:36:30,445 -  INFO  	: my_logger.custom_logger - Selecting features from Dataset


c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.7094681626659987